## Extract Title and Authors from scientific articles

The input is the folder and name of the pdf file. We first extract the text from this file and find the DOI. 

Using the DOI we do extract the bibliographical data from the web.

In [1]:
import pathlib, fitz
import requests
import random

paper_number = random.randint(0,18)
print(paper_number)

# We extract the full text
fname = 'examples/paper_'+str(paper_number)+'.pdf'  # get document filename
with fitz.open(fname) as doc:  # open document
    text = "".join([page.get_text() for page in doc])
# write as a binary file to support non-ASCII characters
pathlib.Path(fname + ".txt").write_bytes(text.encode()) 


8


56610

In [2]:
# Function to extract title and author
def get_title_author(text: str) -> list:
    """
    Function to get the title and author from scientific papers.
    The title and authors are stored in a list

    Args:
    text (str): the text from the article.
        
    Returns:
    list: two entries, one is the title and the second is the name of the authors.

    """
    doi_index = text.find('doi')
    if doi_index == -1:
      doi_index = text.find('DOI')
    if doi_index == -1:
      doi_index == text.find('Doi')  

    doi_start = text.find('10', doi_index)  
    end_space = text.find(' ', doi_start)
    end_line = text.find('\n', doi_start)
    if  end_space < end_line:
      doi_end = end_space
    else:
      doi_end = end_line  
    doi_id = text[doi_start:doi_end]

    # If the DOI is cut we do a second search:
    if len(doi_id)<10:
      doi_index2 = text.find('doi', doi_index+1)
      doi_start = text.find('10', doi_index2)  
      end_space = text.find(' ', doi_start)
      end_line = text.find('\n', doi_start)
      if  end_space < end_line:
        doi_end = end_space
      else:
        doi_end = end_line  
    doi_id = text[doi_start:doi_end]   
    base_url = f"https://doi.org/{doi_id}"
    headers = {"Accept": "text/bibliography; style=bibtex"}
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:
      bibtex = response.text.strip()
    else:
      print(response.status_code)
      print(response.text)
      return None

    title_init = bibtex.find('title')
    title_end = bibtex.find('}', title_init)
    author_init = bibtex.find('author')
    author_end = bibtex.find('}', author_init)
  
    title = bibtex[title_init:title_end+1]
    authors = bibtex[author_init:author_end+1]
    return [title, authors]

In [3]:
get_title_author(text)

['title={PI3K inhibitors: review and new strategies}',
 'author={Zhang, Mingzhen and Jang, Hyunbum and Nussinov, Ruth}']